# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [1]:
# !apt-get update && apt-get install -y python3-opencv

In [2]:
# !pip install opencv-python

In [3]:
# !pip install sklearn

In [4]:
# !pip install pandas

In [5]:
# !pip install efficientnet_pytorch

In [6]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

## Set Arguments & hyperparameters

In [7]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [8]:
os.chdir("/USER/daeyeong")  # 기준 경로 변경

In [9]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [10]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


## Model

In [12]:
import timm 

class Efficientnet(nn.Module):
    def __init__(self, num_classes):
        super(Efficientnet, self).__init__()
        self.efficientnet = timm.create_model('efficientnetv2_s', pretrained=False, num_classes=num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.efficientnet(x)
        
        output = self.softmax(x)
        
        return output

## Utils
### EarlyStopper

In [13]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [14]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [15]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [16]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [21]:
# Load Model
model = Efficientnet(NUM_CLS)
model = model.to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

### epoch 단위 학습 진행

In [22]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0% 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 2.6705256369378834, Acc: 0.47676419965576594, F1-Macro: 0.47545735329056793


  3% 1/30 [00:46<22:27, 46.47s/it]

Epoch 0, Val loss: 2.9124706387519836, Acc: 0.49230769230769234, F1-Macro: 0.32989690721649484
Epoch 1, Train loss: 1.6778925789727106, Acc: 0.6058519793459552, F1-Macro: 0.559756731090574
Epoch 1, Val loss: 1.2350411415100098, Acc: 0.5538461538461539, F1-Macro: 0.46886446886446886
Validation loss decreased 2.9124706387519836 -> 1.2350411415100098


  7% 2/30 [01:32<21:38, 46.39s/it]

...

Epoch 22, Train loss: 0.8472140005065335, Acc: 0.8416523235800344, F1-Macro: 0.8413585962912838
Epoch 22, Val loss: 0.9939139895141125, Acc: 0.6153846153846154, F1-Macro: 0.6094688776736361
Early stopping counter 9/10


 77% 23/30 [23:13<07:56, 68.09s/it]

Epoch 23, Train loss: 1.0906000840995047, Acc: 0.8382099827882961, F1-Macro: 0.8381979996207812


 77% 23/30 [24:22<07:25, 63.58s/it]

Epoch 23, Val loss: 0.9760261229967, Acc: 0.6153846153846154, F1-Macro: 0.6094688776736361
Early stopping counter 10/10
Early stopped


## Inference
### 모델 로드

In [23]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [24]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [25]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [26]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:02,  2.94s/it]

tensor([[8.7028e-01, 1.2972e-01],
        [5.0053e-01, 4.9947e-01],
        [8.7037e-01, 1.2963e-01],
        [5.8376e-01, 4.1624e-01],
        [9.5099e-01, 4.9007e-02],
        [4.0483e-01, 5.9517e-01],
        [2.5888e-01, 7.4112e-01],
        [8.0180e-01, 1.9820e-01],
        [9.1048e-01, 8.9516e-02],
        [1.0000e+00, 6.1822e-14],
        [9.5605e-01, 4.3953e-02],
        [9.2850e-01, 7.1499e-02],
        [9.6480e-01, 3.5196e-02],
        [3.2707e-02, 9.6729e-01],
        [9.8890e-01, 1.1100e-02],
        [3.1130e-01, 6.8870e-01],
        [9.6052e-01, 3.9477e-02],
        [9.8902e-01, 1.0979e-02],
        [2.0691e-02, 9.7931e-01],
        [9.9094e-01, 9.0649e-03],
        [9.8683e-01, 1.3173e-02],
        [9.0489e-01, 9.5114e-02],
        [9.7261e-01, 2.7390e-02],
        [9.4829e-01, 5.1708e-02],
        [1.9092e-03, 9.9809e-01],
        [9.6065e-01, 3.9347e-02],
        [9.0626e-01, 9.3740e-02],
        [1.9761e-05, 9.9998e-01],
        [1.0000e+00, 0.0000e+00],
        [6.457

2it [00:05,  2.96s/it]

tensor([[9.9531e-01, 4.6929e-03],
        [9.5176e-01, 4.8241e-02],
        [9.7149e-01, 2.8505e-02],
        [6.5662e-01, 3.4338e-01],
        [1.0000e+00, 7.1489e-34],
        [9.7620e-01, 2.3798e-02],
        [7.2803e-01, 2.7197e-01],
        [9.5605e-01, 4.3953e-02],
        [4.5689e-01, 5.4311e-01],
        [7.6770e-01, 2.3230e-01],
        [9.5220e-01, 4.7804e-02],
        [2.5948e-04, 9.9974e-01],
        [6.8384e-01, 3.1616e-01],
        [9.2290e-01, 7.7102e-02],
        [9.8440e-01, 1.5599e-02],
        [1.8958e-01, 8.1042e-01],
        [2.8540e-01, 7.1460e-01],
        [9.6881e-01, 3.1191e-02],
        [8.4252e-01, 1.5748e-01],
        [1.4649e-01, 8.5351e-01],
        [3.3583e-01, 6.6417e-01],
        [7.4505e-01, 2.5495e-01],
        [5.5839e-01, 4.4161e-01],
        [8.5269e-01, 1.4731e-01],
        [9.7148e-01, 2.8519e-02],
        [4.6778e-01, 5.3222e-01],
        [5.7964e-02, 9.4204e-01],
        [4.2479e-01, 5.7521e-01],
        [1.2765e-03, 9.9872e-01],
        [9.965

3it [00:08,  2.94s/it]

tensor([[7.7336e-01, 2.2664e-01],
        [1.0000e+00, 1.4621e-12],
        [9.9997e-01, 2.5687e-05],
        [9.8880e-01, 1.1203e-02],
        [3.6272e-01, 6.3728e-01],
        [9.8270e-01, 1.7298e-02],
        [9.8529e-01, 1.4715e-02],
        [9.9843e-01, 1.5689e-03],
        [4.4651e-01, 5.5349e-01],
        [9.3896e-01, 6.1039e-02],
        [5.1044e-04, 9.9949e-01],
        [5.0181e-01, 4.9819e-01],
        [9.8657e-01, 1.3426e-02],
        [1.0000e+00, 0.0000e+00],
        [1.6971e-02, 9.8303e-01],
        [4.2537e-02, 9.5746e-01],
        [1.0000e+00, 2.5325e-06],
        [2.9399e-03, 9.9706e-01],
        [9.9405e-01, 5.9477e-03],
        [4.8191e-01, 5.1809e-01],
        [8.9052e-01, 1.0948e-01],
        [8.5894e-01, 1.4106e-01],
        [9.1882e-01, 8.1180e-02],
        [7.1335e-01, 2.8665e-01],
        [9.6832e-01, 3.1679e-02],
        [8.7514e-01, 1.2486e-01],
        [6.4790e-05, 9.9994e-01],
        [9.1149e-01, 8.8509e-02],
        [1.1217e-01, 8.8783e-01],
        [1.359

4it [00:09,  2.31s/it]

tensor([[7.8015e-06, 9.9999e-01],
        [3.4716e-01, 6.5284e-01],
        [3.7142e-02, 9.6286e-01],
        [9.9598e-01, 4.0192e-03]], device='cuda:0')


### 결과 저장

In [27]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)

In [28]:
df

,file_name,COVID
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,0
4,4.png,0
...,...,...
95,95.png,0
96,96.png,1
97,97.png,1
98,98.png,1
